In [1]:
import math
import numpy as np
import pandas as pd
import datetime
import scipy.stats as stats
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr

In [2]:
# initial derivative parameters
S = 101.15          #stock price
K = 98.01           #strike price
vol = 0.0991        #volatility (%)
r = 0.01            #risk-free rate (%)
N = 10              #number of time steps
M = 1000            #number of simulations

market_value = 3.86 #market price of option
T = ((datetime.date(2022,3,17)-datetime.date(2022,1,17)).days+1)/365    #time in years
print(T)

0.1643835616438356


In [3]:
dt = T/N
mudt = (r*0.5*vol**2)*dt
volsigma = vol*np.sqrt(dt)
lnS = np.log(S)


# Standard Error Placeholders
sum_CT = 0
sum_CT2 = 0
for i in range(M):
    lnSt1 = lnS
    lnSt2  = lnS
    for j in range(N):
        epsilon = np.random.normal()
        lnSt1 = lnSt1 + mudt + volsigma*epsilon
        lnSt2 = lnSt2 + mudt - volsigma*epsilon
    
    ST1 = np.exp(lnSt1)
    ST2 = np.exp(lnSt2)
    CT = 0.5*(max(0, ST1 - K)+max(0, ST2 - K))
    
    sum_CT = sum_CT+ CT
    sum_CT2 = sum_CT2 + CT*CT
    
#finding expectation and SE

C0 = np.exp(-r*T)*sum_CT/M
sigma = np.sqrt( (sum_CT2 - sum_CT*sum_CT/M)*np.exp(-2*r*T) / (M-1) )
SE = sigma/np.sqrt(M)

print("Call value is ${0} with SE +/- {1}".format(np.round(C0,2),np.round(SE,2)))

Call value is $3.73 with SE +/- 0.03


In [4]:
#Faster Method
#precompute constants
dt = T/N
nudt = (r - 0.5*vol**2)*dt
volsdt = vol*np.sqrt(dt)
lnS = np.log(S)

# Monte Carlo Method
Z = np.random.normal(size=(N, M))
delta_lnSt1 = nudt + volsdt*Z
delta_lnSt2 = nudt - volsdt*Z
lnSt1 = lnS + np.cumsum(delta_lnSt1, axis=0)
lnSt2 = lnS + np.cumsum(delta_lnSt2, axis=0)

# Compute Expectation and SE
ST1 = np.exp(lnSt1)
ST2 = np.exp(lnSt2)
CT = 0.5*(np.maximum(0, ST1[-1] - K) + np.maximum(0,ST2[-1] - K))
C0 = np.exp(-r*T)*np.sum(CT)/M

sigma = np.sqrt( np.sum( (CT - C0)**2) / (M-1) )
SE = sigma/np.sqrt(M)

print("Call value is ${0} with SE +/- {1}".format(np.round(C0,2),np.round(SE,2)))


Call value is $3.76 with SE +/- 0.03
